In [1]:
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter import filedialog

import matplotlib
matplotlib.use("TkAgg")
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import numpy as np
import imageio
import re

In [2]:
#Funcion que transforma de rgb a yiq
def rgb_yiq(img):
    yiq = np.zeros(img.shape)
    yiq[:,:,0] = 0.229*img[:,:,0]+0.587*img[:,:,1]+0.114*img[:,:,2]
    yiq[:,:,1] = 0.595716*img[:,:,0]-0.274453*img[:,:,1]-0.321263*img[:,:,2]
    yiq[:,:,2] = 0.211456*img[:,:,0]-0.522591*img[:,:,1]+0.311135*img[:,:,2]
    yiq[:,:,3] = img[:,:,3]
    return yiq

#Funcion que transforma de yiq a rgb
def yiq_rgb(img):
    rgb = np.zeros(img.shape)
    rgb[:,:,0] = np.clip(img[:,:,0]+0.9663*img[:,:,1]+0.6210*img[:,:,2],0,1)
    rgb[:,:,1] = np.clip(img[:,:,0]-0.2721*img[:,:,1]-0.6474*img[:,:,2],0,1)
    rgb[:,:,2] = np.clip(img[:,:,0]-1.1070*img[:,:,1]+1.7046*img[:,:,2],0,1)
    rgb[:,:,3] = img[:,:,3]
    return rgb

#Funcion que modifica la imagen con los parametros "a" para la luminancia, y "b" para la crominancia
def cambio(img,a,b):
    cambio = np.zeros(img.shape)
    cambio[:,:,0] = np.clip(a*img[:,:,0],0.,1.)
    cambio[:,:,1] = np.clip(b*img[:,:,1],-0.5957,0.5957)
    cambio[:,:,2] = np.clip(b*img[:,:,2],-0.5226,0.5226)
    cambio[:,:,3] = img[:,:,3]
    return cambio

#Funcion que determina si se ingreso un numero, ya sea decimal o entero
def es_decimal(cadena):
    patron = r'^[0-9]+(\.[0-9])$'
    regex = re.compile(patron)
    return bool(regex.search(cadena))

In [3]:
window = Tk()
#Construccion de la ventana
window.title("Trabajos")
window.geometry('1100x950')
tab_control = ttk.Notebook(window)
tab1 = ttk.Frame(tab_control)
tab2 = ttk.Frame(tab_control)
tab3 = ttk.Frame(tab_control)
tab_control.add(tab1, text='Tp 1')
tab_control.add(tab2, text='Tp 2')
tab_control.add(tab3, text='Tp 3')

#Funcionalidades del Trabajo Practico 1

#Funcion click del boton "Procesar Imagen". Aplica todas las funciones necesarias para modificar la imagen y graficael resultado en la ventana
def clicked(image,a,b):
    if es_decimal(a.get()) or a.get().isnumeric() and es_decimal(b.get()) or b.get().isnumeric() :    #Comprueba si se ingreso un numero para los parametros "a" y "b"
        img = rgb_yiq(image)
        img = cambio(img,float(a.get()),float(b.get()))
        img = yiq_rgb(img)
        fig2 = plt.figure(figsize=(5,5))
        plt.axis('off')
        plt.imshow(img) 
        canvas = FigureCanvasTkAgg(fig2, master = tab1)
        canvas.draw()
        canvas.get_tk_widget().grid(padx=15,pady=15,column=2,row=1)
    else:
        messagebox.showinfo('Error', 'Debe ingresar valores numericos')

#Funcion click del boton "Abrir Imagen". Carga la imagen y la grafica en la ventana
def clickedopen():
    image = np.clip(imageio.imread(filedialog.askopenfilename())/255.,0.,1.)
    fig = plt.figure(figsize=(5,5))
    plt.axis('off')
    plt.imshow(image) 
    canvas = FigureCanvasTkAgg(fig, master = tab1)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=15,column=0,row=1,columnspan=2)
    
    #Se crean las estructuras necesarias para cargar los valores de "a" y "b"
    lbl3 = Label(tab1, text = 'Ingrese "a" (varia luminancia)')
    lbl3.grid(pady = 15, row = 3,column = 0)
    lbl4 = Label(tab1, text = 'Ingrese "b" (varia crominancia)')
    lbl4.grid(pady = 15, row = 4,column = 0)

    a = Entry(tab1,width=10)
    a.grid(padx = 10,row=3, column=1)
    b = Entry(tab1,width=10)
    b.grid(padx = 10,row=4, column=1)
    
    #Se crea el boton "procesar Imagen"
    btn = Button(tab1, text="Procesar Imagen", width=50, command=lambda: clicked(image,a,b))                             
    btn.grid(pady=15,column=0,row=5,columnspan=2)
        
#Definicion de los Labels y boton que se muestran inicialmente
lbl1 = Label(tab1, text= 'Imagen Original',font=("Arial Bold",18))
lbl1.grid(column=0, row=0, columnspan=2)

lbl2 = Label(tab1, text= 'Imagen Modificada', font=("Arial Bold",18),width=30)
lbl2.grid(column=2, row=0) 

btnopen = Button(tab1, text="Abrir Imagen", width=50, command=lambda: clickedopen())                               
btnopen.grid(pady=15,column=0,row=2,columnspan=2)


In [4]:
#Funciones del TP2
#Funcion del clampeo en RGB
def clamp_rgb(img1,img2):
    rgb = np.zeros(img1.shape)
    rgb[:,:,0] = np.clip(img1[:,:,0]+img2[:,:,0],0.,1.)
    rgb[:,:,1] = np.clip(img1[:,:,1]+img2[:,:,1],0.,1.)
    rgb[:,:,2] = np.clip(img1[:,:,2]+img2[:,:,2],0.,1.)
    rgb[:,:,3] = np.clip(img1[:,:,3]+img2[:,:,3],0.,1.)
    return rgb

#Funcion del operador promedio en RGB
def promedio_rgb(img1,img2):
    rgb = np.zeros(img1.shape)
    rgb[:,:,0] = (img1[:,:,0]+img2[:,:,0])/2
    rgb[:,:,1] = (img1[:,:,1]+img2[:,:,1])/2
    rgb[:,:,2] = (img1[:,:,2]+img2[:,:,2])/2
    rgb[:,:,3] = (img1[:,:,3]+img2[:,:,3])/2
    return rgb

#Funcion del clampeo en YIQ
def clamp_yiq(img1,img2):
    yiq1 = rgb_yiq(img1)
    yiq2 = rgb_yiq(img2)
    yiq = np.zeros(yiq1.shape)
    yiq[:,:,0] = np.clip(yiq1[:,:,0]+yiq2[:,:,0],0.,1.)
    yiq[:,:,1] = yiq1[:,:,1]
    yiq[:,:,2] = yiq1[:,:,2]
    yiq[:,:,3] = yiq1[:,:,3]
    return yiq_rgb(yiq) 

#Funcion del operador promedio en YIQ
def promedio_yiq(img1,img2):
    img1 = rgb_yiq(img1)
    img2 = rgb_yiq(img2)
    yiq = np.zeros(img1.shape)
    yiq[:,:,0] = (img1[:,:,0]+img2[:,:,0])/2
    yiq[:,:,1] = (img1[:,:,1])
    yiq[:,:,2] = (img1[:,:,2])
    yiq[:,:,3] = (img1[:,:,3])
    return yiq_rgb(yiq)

#Funcion resta en YIQ
def resta_yiq(img1,img2):
    img1 = rgb_yiq(img1)
    img2 = rgb_yiq(img2)
    yiq = np.zeros(img1.shape)
    yiq[:,:,0] = np.clip(img1[:,:,0]-img2[:,:,0],0.,1.)
    yiq[:,:,1] = img1[:,:,1]
    yiq[:,:,2] = img1[:,:,2]
    yiq[:,:,3] = img1[:,:,3]
    return yiq_rgb(yiq)

#Definicion de la funcion if_lighter, se procesa en YIQ
def if_lighter(img1,img2):
    img1 = rgb_yiq(img1)
    img2 = rgb_yiq(img2)
    yiq = np.zeros(img1.shape)
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            if img1[i,j,0] > img2[i,j,0]:
                yiq[i,j,0] = img1[i,j,0]
                yiq[i,j,1] = img1[i,j,1]
                yiq[i,j,2] = img1[i,j,2]
                yiq[i,j,3] = img1[i,j,3]
            else:
                yiq[i,j,0] = img2[i,j,0]
                yiq[i,j,1] = img2[i,j,1]
                yiq[i,j,2] = img2[i,j,2]
                yiq[i,j,3] = img2[i,j,3]
    return yiq_rgb(yiq)

#Definicion de la funcion if_darker, se procesa en YIQ
def if_darker(img1,img2):
    img1 = rgb_yiq(img1)
    img2 = rgb_yiq(img2)
    yiq = np.zeros(img1.shape)
    for i in range(img1.shape[0]):
        for j in range(img1.shape[1]):
            if img1[i,j,0] < img2[i,j,0]:
                yiq[i,j,0] = img1[i,j,0]
                yiq[i,j,1] = img1[i,j,1]
                yiq[i,j,2] = img1[i,j,2]
                yiq[i,j,3] = img1[i,j,3]
            else:
                yiq[i,j,0] = img2[i,j,0]
                yiq[i,j,1] = img2[i,j,1]
                yiq[i,j,2] = img2[i,j,2]
                yiq[i,j,3] = img2[i,j,3]
    return yiq_rgb(yiq)

In [5]:
#Configuracion de ventana de TP2
#Funcion click del boton "Procesar Imagen", se procesa el operador de acuerdo a la eleccion del usuario
def clicked2(image1, image2):
    if combo.get() == 'Clamp RGB':
        res = clamp_rgb(image1,image2)
    elif combo.get() == 'Clamp YIQ':
        res = clamp_yiq(image1,image2)
    elif combo.get() == 'Promedio RGB':
        res = promedio_rgb(image1,image2)
    elif combo.get() == 'Promedio YIQ':
        res = promedio_yiq(image1,image2)
    elif combo.get() == 'Resta YIQ':
        res = resta_yiq(image1,image2)
    elif combo.get() == 'If Lighter':
        res = if_lighter(image1,image2)
    else:
        res = if_darker(image1,image2)
    
    fig3_2 = plt.figure(figsize=(5,3))        #Se grafica la imaen resultante
    plt.axis('off')
    plt.imshow(res) 
    canvas = FigureCanvasTkAgg(fig3_2, master = tab2)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=5,column=0,row=5,columnspan = 3)
    
    lbl_n = Label(tab2, text= 'Imagen Combinada',font=("Arial Bold",18))
    lbl_n.grid(pady=5,column=0, row=4, columnspan=3)
        
#Definicion de labels iniciales        
lbl1_2 = Label(tab2, text= 'Imagen 1',font=("Arial Bold",18))
lbl1_2.grid(column=0, row=0, columnspan=2)

lbl2_2 = Label(tab2, text= 'Imagen 2', font=("Arial Bold",18))
lbl2_2.grid(column=2, row=0) 

#Las imagenes en este caso estan precargadas, debido a que debian poseer la misma resolucion
image2 = np.clip(imageio.imread('foto1.bmp')/255.,0.,1.)

fig1_2 = plt.figure(figsize=(5,3))
plt.axis('off')
plt.imshow(image2) 

canvas = FigureCanvasTkAgg(fig1_2, master = tab2)
canvas.draw()
canvas.get_tk_widget().grid(padx=15,pady=15,column=0,row=1,columnspan=2)

image3 = np.clip(imageio.imread('foto2.bmp')/255.,0.,1.)

fig2_2 = plt.figure(figsize=(5,3))
plt.axis('off')
plt.imshow(image3) 

canvas = FigureCanvasTkAgg(fig2_2, master = tab2)
canvas.draw()
canvas.get_tk_widget().grid(padx=15,pady=15,column=2,row=1)

#Se define un Combobox, para que el usuario pueda seleccionar el operador que desea aplicar
combo = ttk.Combobox(tab2, state="readonly")
combo['values'] = ('Clamp RGB','Clamp YIQ','Promedio RGB', 'Promedio YIQ', 'Resta YIQ', 'If Lighter', 'If Darker')
combo.current(0)
combo.grid(padx = 15, pady = 15, column = 1, row = 2)

lbl3_2 = Label(tab2, text = 'Seleccione operador de imagen:')
lbl3_2.grid(pady = 15, row = 2,column = 0)

btn2 = Button(tab2, text="Procesar Imagen", width=50, command=lambda: clicked2(image2, image3))                               #state='disabled'
btn2.grid(pady=15,column=0,row=3,columnspan=3)


In [6]:
#Funciones Trabajo Practico 3
#Funcion que grafica el Histograma de una Imagen en la Ventana
def histograma(img):
    yiq = rgb_yiq(img)
    fig2_3 = plt.figure(figsize=(5,3))
    plt.hist(yiq[:,:,0].reshape(img.shape[0]*img.shape[1]), bins = 10)
    #plt.show()
    canvas = FigureCanvasTkAgg(fig2_3, master = tab3)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=15,column=2,row=1,columnspan=2)

#Funcion que aplica el operador raiz cuadrada a una imagen
def raiz_2(img):   
    yiq = rgb_yiq(img)
    yiq2 = np.zeros(yiq.shape)
    yiq2[:,:,0] = yiq[:,:,0]**(1./2)
    yiq2[:,:,1] = yiq[:,:,1]
    yiq2[:,:,2] = yiq[:,:,2]
    yiq2[:,:,3] = yiq[:,:,3]
    return yiq_rgb(yiq2)

#Funcion que aplica el operador raiz cubica a una imagen
def raiz_3(img):   
    yiq = rgb_yiq(img)
    yiq2 = np.zeros(yiq.shape)
    yiq2[:,:,0] = yiq[:,:,0]**(1./3)
    yiq2[:,:,1] = yiq[:,:,1]
    yiq2[:,:,2] = yiq[:,:,2]
    yiq2[:,:,3] = yiq[:,:,3]
    return yiq_rgb(yiq2)

#Funcion que aplica el operador cuadratico a una imagen
def cuadratica(img):   
    yiq = rgb_yiq(img)
    yiq2 = np.zeros(yiq.shape)
    yiq2[:,:,0] = yiq[:,:,0]**2
    yiq2[:,:,1] = yiq[:,:,1]
    yiq2[:,:,2] = yiq[:,:,2]
    yiq2[:,:,3] = yiq[:,:,3]
    return yiq_rgb(yiq2)

#Funcion que aplica el operador cubico a una imagen
def cubo(img):   
    yiq = rgb_yiq(img)
    yiq2 = np.zeros(yiq.shape)
    yiq2[:,:,0] = yiq[:,:,0]**3
    yiq2[:,:,1] = yiq[:,:,1]
    yiq2[:,:,2] = yiq[:,:,2]
    yiq2[:,:,3] = yiq[:,:,3]
    return yiq_rgb(yiq2)

#Funcion que aplica Lineal a Trozos
def lineal_tr(img,combo3):
    yiq = rgb_yiq(img)
    yiq2 = np.zeros(yiq.shape)
    aux = float(combo3.get())     #Se toma el limite seleccionado por el usuario para aplicar Lineal a Trozos
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if yiq[i,j,0] < aux:      #Valores menores que el limite, se transforman a 0
                yiq2[i,j,0] = 0
                yiq2[i,j,1] = yiq[i,j,1]
                yiq2[i,j,2] = yiq[i,j,2]
                yiq2[i,j,3] = yiq[i,j,3]
            elif yiq[i,j,0] > (1-aux):      #Valores mayores que "1 menos el limite" se transforman a 1
                yiq2[i,j,0] = 1
                yiq2[i,j,1] = yiq[i,j,1]
                yiq2[i,j,2] = yiq[i,j,2]
                yiq2[i,j,3] = yiq[i,j,3]
            else:
                yiq2[i,j,0] = (1/((1-aux)-aux))*(yiq[i,j,0]-aux)    #Se aplica Interpolacion
                yiq2[i,j,1] = yiq[i,j,1]
                yiq2[i,j,2] = yiq[i,j,2]
                yiq2[i,j,3] = yiq[i,j,3]
    return yiq_rgb(yiq2)

In [ ]:
#Configuracion de ventana de TP3
#Funcion CLick del boton "Procesar", se ejecuta el operador seleccionado por el usuario
def clicked3(img,combo2,combo3):
    if combo2.get() == 'Cuadratica':
        res2 = cuadratica(img)
    elif combo2.get() == 'Cubo':
        res2 = cubo(img)
    elif combo2.get() == 'Raiz cuadrada':
        res2 = raiz_2(img)
    elif combo2.get() == 'Raiz Cubo':
        res2 = raiz_3(img)
    else:
        res2 = lineal_tr(img,combo3)
    
    #Se muestra en la ventana la imagen resultante
    fig3_3 = plt.figure(figsize=(5,3))
    plt.axis('off')
    plt.imshow(res2) 
    canvas = FigureCanvasTkAgg(fig3_3, master = tab3)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=5,column=0,row=6,columnspan = 2)
    
    lbl_n = Label(tab3, text= 'Imagen Resultante',font=("Arial Bold",18))
    lbl_n.grid(pady=5,column=0, row=5, columnspan=2)
    
    #Se muestra el Histograma de la imagen resultante
    yiq = rgb_yiq(res2)
    fig4_3 = plt.figure(figsize=(5,3))
    plt.hist(yiq[:,:,0].reshape(res2.shape[0]*res2.shape[1]), bins = 10)
    #plt.show()
    canvas = FigureCanvasTkAgg(fig4_3, master = tab3)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=5,column=2,row=6,columnspan=2)

#Funcion CLick del boton "Abrir Imagen", carga la imagen y la muestra en la ventana junto con su histograma
def clickedopen3():
    img = np.clip(imageio.imread(filedialog.askopenfilename())/255.,0.,1.)
    
    fig1_3 = plt.figure(figsize=(5,3))
    plt.axis('off')
    plt.imshow(img) 

    canvas = FigureCanvasTkAgg(fig1_3, master = tab3)
    canvas.draw()
    canvas.get_tk_widget().grid(padx=15,pady=15,column=0,row=1,columnspan=2)

    histograma(img)
    
    #Se define un Combobox para que el usuario seleccione el operador que desea
    combo2 = ttk.Combobox(tab3, state="readonly")
    combo2['values'] = ('Cuadratica','Cubo','Raiz cuadrada', 'Raiz Cubo', 'Lineal a Trozos')
    combo2.current(0)
    combo2.grid(padx = 15, pady = 15, column = 1, row = 3)

    #Se define un segundo Combobox que solo opera con el operador "Lineal a trozos", este determina el valor de los limites
    combo3 = ttk.Combobox(tab3, state="readonly")
    combo3['values'] = (0.1,0.2,0.3,0.4)
    combo3.current(0)
    combo3.grid(padx = 15, pady = 15, column = 3, row = 3)

    lbl3_3 = Label(tab3, text = 'Seleccione operador de imagen:')
    lbl3_3.grid(pady = 15, row = 3,column = 0)
    
    lbl4_3 = Label(tab3, text = 'Para Linel a trozos:')
    lbl4_3.grid(pady = 15, row = 3,column = 2)

    btn3 = Button(tab3, text="Procesar", width=50, command=lambda: clicked3(img,combo2,combo3))                               #state='disabled'
    btn3.grid(pady=15,column=0,row=4,columnspan=4)


lbl1_3 = Label(tab3, text= 'Imagen Original',font=("Arial Bold",18))
lbl1_3.grid(column=0, row=0, columnspan=2)

lbl2_3 = Label(tab3, text= 'Histograma', font=("Arial Bold",18))
lbl2_3.grid(column=2, row=0, columnspan=2) 


btnopen3 = Button(tab3, text="Abrir Imagen", width=50, command=lambda: clickedopen3())                               #state='disabled'
btnopen3.grid(pady=15,column=0,row=2,columnspan=2)


tab_control.pack(expand=1, fill='both')
window.mainloop()

